In [17]:
from jmetal.algorithm.multiobjective.nsgaii import NSGAII
from jmetal.operator import SBXCrossover, PolynomialMutation
from jmetal.problem import ZDT1
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.core.problem import FloatProblem
from jmetal.core.solution import FloatSolution

## Defining the problem

In [18]:

class o(FloatProblem):
    profit_values=[]
    cvar_values=[]
    def __init__(self):
        super(o, self).__init__()
        self.number_of_variables = 4
        self.number_of_objectives = 2
        self.number_of_constraints = 1

        self.obj_directions = [self.MINIMIZE,self.MINIMIZE]
        self.obj_labels = ['f(x)', 'f(y)']

        # Constraints
        self.lower_bound = [0,0,0,0.15]
        self.upper_bound = [0.5,0.5,0.15,0.5]
        
    #['home_lons','auto_loans','card_loans','bussiness_loans']
    
    def evaluate(self, solution: FloatSolution) -> FloatSolution:
        #[8.96,7.75,8.21,4.58]
        # profit percentile
        profit=[0.0896,0.075,0.0821,0.0458]
        
        # budget
        max_value=1000
        #[29.2,20.2,20.7,39.5]
        # cvar percentile
        cvar=[0.292,0.202,0.207,0.395]
        
        profit_score=0
        cvar_score=0
        
        q=0
#         print(solution)
        sum_of_variables=sum(solution.variables)
        for i in solution.variables:
            i=i/(sum_of_variables)
            profit_score+=((max_value*i)*profit[q])
            cvar_score+=(cvar[q]*(i*max_value))**2   
            q+=1
        
        cvar_score=((0.25*cvar_score)**(0.5))
        o.profit_values.append(profit_score)
        o.cvar_values.append(cvar_score)
#         print("cvar---->",cvar)
#         print("profit---->",profit)
  
        solution.objectives[0]=1/profit_score # maximising the return
        solution.objectives[1]=cvar_score     # minimising the cvar
        
        self.__evaluate__constraints(solution) # constraints
        return solution
    def get_name(self):
        return 'Tanaka'
    def __evaluate__constraints(self,sol):
        sol.constraints.append(1/1000*(sum(sol.variables)-1)) 
        
        
        
       

In [19]:
import time
problem = o()
max_evaluations = 3000
start_time=time.time()
algorithm = NSGAII(
    problem=problem,
    population_size=40,
    offspring_population_size=40,
    mutation=PolynomialMutation(probability=1.0 / problem.number_of_variables, distribution_index=20),
    crossover=SBXCrossover(probability=1.0, distribution_index=20),
    termination_criterion=StoppingByEvaluations(max_evaluations)
)

algorithm.run()
solutions = algorithm.get_result()
end_time=time.time()
print(abs(start_time-end_time))

2.2242960929870605


In [20]:
data={"home_loans":[],"automobile":[],"card_loans":[],"bussiness_loans":[],"profit":[],"cvar":[],"total_budget_used":[]}
for i in solutions:
    variables=list(i.variables)
    sum_of_variables=sum(variables)
    for i in range(len(variables)):
        variables[i]=variables[i]/sum_of_variables
    sum_of_variables=sum(variables)
    o3=0
    ##  change
    l=[0.0896,0.075,0.0821,0.0458]
    cs=[0.292,0.202,0.207,0.395]
    o1=0
    o4=0
    data['home_loans'].append(variables[0]/sum_of_variables)
    data['automobile'].append(variables[1]/sum_of_variables)
    data['card_loans'].append(variables[2]/sum_of_variables)
    data['bussiness_loans'].append(variables[3]/sum_of_variables)
    data['total_budget_used'].append(sum_of_variables)
    for j in variables:
        o3+=(j*1000)*l[o1]
        o4+=((j*1000)*cs[o1])**2
        o1+=1
    o4=((0.25*o4)**(0.5))
    data["profit"].append(o3)
    data["cvar"].append(o4)
    print(o3,o4)
    print()

78.96687416956462 82.57138834045621

75.13160180040249 67.09858586298692

78.43772575412505 76.0232958550423

76.95328253002623 69.99350043300451

77.50868763517761 71.21883215321762

76.24387316305373 68.35120583019261

77.87433457996534 72.34210341310727

78.90954832185426 81.7844853738569

78.52063002489268 76.89229067992586

75.51343761491026 67.45439054539465

75.773196536291 67.72078439018999

77.28010573353171 70.57980109182338

76.03179802035544 68.042133728899

76.74018968132232 69.41536598824999

77.67619689893407 71.86664463307685

78.17716769633044 73.7432914935431

77.94440928933403 72.60752904304341

77.27511785327961 70.57530022684665

78.56381974258437 77.62079970517999

77.07022253594958 70.08604399663915

78.86057185884927 81.14421720891252

78.63628958361983 78.28245979809142

78.36341625429696 75.44349267105707

78.29295639371036 74.67219195704833

75.28749151313271 67.17886998826465

78.83939822616762 80.77709287856105

75.97791039664463 67.96973861355265

75.82557

In [21]:
import pandas as pd
data1=pd.DataFrame(data)


data1

,home_loans,automobile,card_loans,bussiness_loans,profit,cvar,total_budget_used
0,0.498006,0.203186,0.149343,0.149464,78.966874,82.571388,1.0
1,0.236043,0.463993,0.149981,0.149983,75.131602,67.098586,1.0
2,0.431468,0.309605,0.129448,0.129479,78.437726,76.023296,1.0
3,0.334378,0.402058,0.131335,0.132230,76.953283,69.993500,1.0
4,0.355059,0.403058,0.120876,0.121007,77.508688,71.218832,1.0
5,0.287484,0.445245,0.133634,0.133637,76.243873,68.351206,1.0
6,0.374622,0.390654,0.117322,0.117402,77.874335,72.342103,1.0
7,0.491060,0.214118,0.147352,0.147470,78.909548,81.784485,1.0
8,0.441929,0.292871,0.132570,0.132629,78.520630,76.892291,1.0
9,0.250367,0.465400,0.142085,0.142148,75.513438,67.454391,1.0


In [22]:
# maximum profit
data1[data1['profit']==data1['profit'].max()]

,home_loans,automobile,card_loans,bussiness_loans,profit,cvar,total_budget_used
0,0.498006,0.203186,0.149343,0.149464,78.966874,82.571388,1.0


In [23]:
#minimum cvar
data1[data1['cvar']==data1['cvar'].min()]

,home_loans,automobile,card_loans,bussiness_loans,profit,cvar,total_budget_used
1,0.236043,0.463993,0.149981,0.149983,75.131602,67.098586,1.0


In [24]:
pareto_profit=list(data1['profit'])
pareto_cvar=list(data1['cvar'])
min_values=[]
for i in range(len(pareto_profit)):
    m=-1
    for j in range(3000):
        euclidean_distance=((pareto_profit[i]-o.profit_values[j])**2+(pareto_cvar[i]-o.cvar_values[j])**2)**0.5
        if(m==-1):
            m=euclidean_distance
        elif(euclidean_distance==0):
            continue
        else:
            m=min(m,euclidean_distance)
    min_values.append(m)

print((sum(min_values)**0.5)/len(min_values))

0.015954819946440595


In [25]:
#Schott’s spacing metric
min_diff=[]
for i in range(len(o.profit_values)):
    m=-1
    for j in range(40):
        euclidean_distance=((pareto_profit[j]-o.profit_values[i])**2+(pareto_cvar[j]-o.cvar_values[i])**2)**0.5
        if(m==-1):
            m=euclidean_distance
        else:
            m=min(m,euclidean_distance)
    min_diff.append(m)
avg=sum(min_diff)/len(min_diff)
n=0
for i in min_diff:
    n+=abs(i-avg)
print(((1/3000)*n)**0.5)

1.1724350565293002


In [14]:
min_values

[0.03443540833663027,
 0.03443540833663027,
 0.02256766023007121,
 0.5190818704349809,
 0.5224299725999797,
 0.09335009762925282,
 0.02084120843908734,
 0.08151059226132767,
 0.1276816090949027,
 1.7800407133146672,
 0.054834622071862876,
 0.6397858653829581,
 0.019713518581071374,
 0.028634843150460942,
 0.027635304760367768,
 0.005750687898085579,
 0.05963071342852546,
 0.13076194623543894,
 0.0039443916298398255,
 0.12527255932507295,
 0.7261417194257737,
 0.013067655120983923,
 0.2834716991612542,
 0.004612685353582339,
 0.026253170171745922,
 0.09032695672691896,
 0.06237326413992839,
 1.3767320443551645,
 1.5172038309636906,
 0.02363674229814916,
 0.05466219486214805,
 0.06638119353030898,
 0.057328163981600094,
 0.05558505564061283,
 0.12219605611095914,
 0.20851398346373157,
 0.02959174140889949,
 0.2708806346177139,
 0.03978769034845437,
 0.36009550021590964]